In [1]:
import pandas as pd
import numpy as np
import anndata as ad

# Metadata

In [2]:
f = '../data_dump/candidate_genes_vincent_0328.csv'
# df0 = pd.read_excel(f, sheet_name=0)
df0 = pd.read_csv(f)

print(df0.shape)

(87, 6)


In [3]:
df0

,Gene,Subclass,Type/Gradient,Function,Priority,Note
0,Baz1a,L2/3,A<B>C,TF,***,NaN
1,Agmat,L2/3,NaN,other,NaN,NaN
2,Adamts2,L2/3,A>B=C,other,***,NaN
3,Rrad,L2/3,NaN,other,NaN,NaN
4,Trpc6,L2/3,A<B>C,IC,***,NaN
...,...,...,...,...,...,...
82,Ptpru,L2/3,all,CAM,NaN,NaN
83,Chrm3,L2/3,all,IC,NaN,NaN
84,Whrn,L4,NaN,other,NaN,NaN
85,Stard8,L2/3,all,other,NaN,NaN


# Files 

In [4]:
f = '../data_dump/P38_glut.h5ad'
adata = ad.read(f)
adata

AnnData object with n_obs × n_vars = 10485 × 30869
    obs: 'batch', 'n_genes', 'percent_mito', 'n_counts', 'leiden', 'Doublet', 'Doublet Score', 'cluster', 'Class_broad', 'sample', 'Age', 'subclass', 'layer', 'Type', 'Subclass'
    var: 'gene_ids', 'feature_types', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: 'Type_colors', 'cluster_colors', "dendrogram_['leiden']", 'hvg', 'layer_colors', 'leiden', 'leiden_colors', 'neighbors', 'pca', 'sample_colors', 'umap'
    obsm: 'X_harmony', 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

# save specific genes

In [5]:
cgenes = np.intersect1d(adata.var.index.values, df0['Gene'].values)
print([g for g in df0['Gene'].values if g not in cgenes])
cgenes.shape

['Neurod1']


(86,)

In [6]:
%%time
adata = adata[:,cgenes]
adata

CPU times: user 128 ms, sys: 75.9 ms, total: 204 ms
Wall time: 202 ms


View of AnnData object with n_obs × n_vars = 10485 × 86
    obs: 'batch', 'n_genes', 'percent_mito', 'n_counts', 'leiden', 'Doublet', 'Doublet Score', 'cluster', 'Class_broad', 'sample', 'Age', 'subclass', 'layer', 'Type', 'Subclass'
    var: 'gene_ids', 'feature_types', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: 'Type_colors', 'cluster_colors', "dendrogram_['leiden']", 'hvg', 'layer_colors', 'leiden', 'neighbors', 'pca', 'sample_colors', 'umap'
    obsm: 'X_harmony', 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [7]:
def rename_clsts(x):
    thedict = {
        'L2/3_A': 'L2/3_B',
        'L2/3_B': 'L2/3_A',
    }
    if x in thedict.keys():
        return thedict[x]
    else:
        return x

In [8]:
%%time
adata = adata[adata.obs['cluster'].str.contains(r'^L2|L4')]
adata.obs['cluster'] = adata.obs['cluster'].apply(rename_clsts) 
adata

Trying to set attribute `.obs` of view, copying.


CPU times: user 163 ms, sys: 55 ms, total: 218 ms
Wall time: 216 ms


AnnData object with n_obs × n_vars = 5837 × 86
    obs: 'batch', 'n_genes', 'percent_mito', 'n_counts', 'leiden', 'Doublet', 'Doublet Score', 'cluster', 'Class_broad', 'sample', 'Age', 'subclass', 'layer', 'Type', 'Subclass'
    var: 'gene_ids', 'feature_types', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: 'Type_colors', 'cluster_colors', "dendrogram_['leiden']", 'hvg', 'layer_colors', 'leiden', 'neighbors', 'pca', 'sample_colors', 'umap'
    obsm: 'X_harmony', 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [9]:
adata.write('/bigstore/GeneralStorage/fangming/projects/visctx/data_dump/P38_glut_genes_v2.h5ad')